In [12]:
from langchain.callbacks.stdout import StdOutCallbackHandler

from langchain import OpenAI
import openai
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_ENDPOINT  = os.environ.get("OPENAI_ENDPOINT")
OPENAI_API_VERSION  = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_TYPE  = os.environ.get("OPENAI_API_TYPE")

openai.api_type = OPENAI_API_TYPE
openai.api_base = OPENAI_ENDPOINT
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY

test_llm = OpenAI(
    # engine = "asada001",
    # engine="aschatgpt35",
    engine="asdavinci003",
    # model_kwargs= {
    #     "engine": "asdavinci003",
    # },
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_ENDPOINT,
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [13]:
services = [
    {
        "SID": "service-1",
        "name": "企业查询",
        "description": "查询企业的基本信息，如成立时间、注册资本、注册地址、经营范围等",
        "params": [
            {
                "name": "vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            }
        ]
    },
    {
        "SID": "service-2",
        "name": "企业间投资路径查询",
        "description": "查询两个企业间的投资路径, 如A企业投资B企业, B企业投资C企业, 那么A企业和C企业间就存在投资路径",
        "params": [
            {
                "name": "from_vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            },
            {
                "name": "to_vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            },
        ]
    },
    {
        "SID": "service-3",
        "name": "企业实控人分析",
        "description": "查询企业的实际控制人，以及实际控制人的股权结构",
        "params": [
            {
                "name": "vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            }
        ]
    }
]


In [15]:
from custom_chains.service_type_chain import ServiceTypeChain

chain = ServiceTypeChain.from_list(
    llm=test_llm,
    service_list=services,
    verbose=True,
    callbacks=[StdOutCallbackHandler()]
)

res = chain({"question": "你能帮我做什么事情"})
print(res)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceabi

In [17]:
from custom_chains.entity_search_chain import EntitySearchChain

chain = EntitySearchChain.from_llm(
    llm=test_llm,
    verbose=True,
    # callbacks=[StdOutCallbackHandler()]
)

test_res = chain({"question": "上海爱数成立于哪一年", "entity_types": ["Enterprise"], "entity_number": 2, "sid": "1"})
print(test_res)



> Entering new  chain...
Prompt after formatting:

    INSTRUCTIONS:
    You are helpful of information extraction. You can extract entities from any sentence. And you can do it in Chinese.
    Please recognize the entities or properties mentioned the in question. You are trying to extract 2 entities.
    
    Entity_types:
    ['Enterprise']
    
    EXAMPLES:
    question:
    {
        "question": "上海爱数和长沙爱数的关系是什么?",
        "entity_number": 2,
        "entity_types": ["Enterprise"],
    }
    answer:
    {
        "entities": [
            {
                "type": "Enterprise",
                "name": "上海爱数"
            },
            {
                "type": "Enterprise",
                "name": "长沙爱数"
            }
       ],
       "message": "I extracted 2 entities. Which are: 上海爱数, 长沙爱数."
    }
    
    If you can't extract all entities, please just extract as many as you can.
    
    DO NOT try to translate question into English. And anwser should be in Chinese.
    
    